<a href="https://colab.research.google.com/github/ShraddhaSharma24/Mental-Health-Companion-Agent/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mental Health Companion Agent - Combined Notebook (main.ipynb)

# ============== Listener Agent ==============
class ListenerAgent:
    def __init__(self):
        self.name = "Listener Agent"

    def listen(self, user_input):
        return {"text": user_input}

# ============== Support Agent ==============
class SupportAgent:
    def __init__(self):
        self.name = "Support Agent"

    def provide_support(self, sentiment, emotions):
        if sentiment == "negative":
            response = "I'm really sorry to hear that you're feeling this way. I'm here for you."
        elif sentiment == "positive":
            response = "That's great to hear! Keep up the positive energy!"
        else:
            response = "I'm here to chat whenever you need."

        if "anxious" in emotions:
            response += " Would you like me to guide you through a quick breathing exercise?"

        return response

# ============== Resource Agent ==============
class ResourceAgent:
    def __init__(self):
        self.name = "Resource Agent"

    def recommend_resources(self, sentiment):
        if sentiment == "negative":
            return ["Mindfulness App", "Talk to a Friend", "Mental Health Helpline"]
        elif sentiment == "positive":
            return ["Journal Your Happiness", "Share Your Positivity", "Motivation Podcast"]
        else:
            return ["General Well-being Tips", "Daily Gratitude Practice"]

# ============== Utils (Sentiment & Emotion Detection) ==============
from transformers import pipeline

# Load sentiment analysis pipeline once
sentiment_pipeline = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

def analyze_sentiment(text):
    result = sentiment_pipeline(text)[0]
    if result['label'] == 'POSITIVE':
        return "positive"
    elif result['label'] == 'NEGATIVE':
        return "negative"
    else:
        return "neutral"

def detect_emotions(text):
    emotions = []
    if "anxious" in text.lower() or "nervous" in text.lower():
        emotions.append("anxious")
    if "happy" in text.lower() or "excited" in text.lower():
        emotions.append("happy")
    if "sad" in text.lower() or "depressed" in text.lower():
        emotions.append("sad")
    return emotions

# ============== Conversation Workflow ==============
def run_conversation_workflow():
    listener = ListenerAgent()
    support = SupportAgent()
    resource = ResourceAgent()

    user_input = input("💬 How are you feeling today? ")

    user_data = listener.listen(user_input)

    sentiment = analyze_sentiment(user_data['text'])
    emotions = detect_emotions(user_data['text'])

    support_response = support.provide_support(sentiment, emotions)
    resources = resource.recommend_resources(sentiment)

    print("\n🤖 Support Response:", support_response)
    print("\n📚 Recommended Resources:", ", ".join(resources))

# ============== Main Entry Point ==============
if __name__ == "__main__":
    run_conversation_workflow()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


💬 How are you feeling today? I'm super excited!

🤖 Support Response: That's great to hear! Keep up the positive energy!

📚 Recommended Resources: Journal Your Happiness, Share Your Positivity, Motivation Podcast


In [3]:
!pip install crewai transformers

In [4]:
!pip install crewai litellm[huggingface] transformers accelerate

In [5]:
from crewai import Agent, Task, Crew
from litellm import completion

In [6]:
pip install transformers accelerate torch

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Fully public model
model_name = "microsoft/phi-2"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"  # Use GPU if available
)

def local_llm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Quick Test
print(local_llm("What is the capital of France?"))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the capital of France?
A) London
B) Paris
C) Rome
D) Berlin
Answer: B) Paris



In [8]:
!pip install crewai transformers accelerate torch

In [9]:
from crewai import Task, Crew
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Local Model - Phi-2
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")

# Local LLM Helper
def local_llm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Custom Agent (Bypass litellm - Direct Local Call)
class LocalAgent:
    def __init__(self, name, role, goal, backstory):
        self.name = name
        self.role = role
        self.goal = goal
        self.backstory = backstory

    def run(self, task_description, inputs=None):
        prompt = f"""
        Role: {self.role}
        Goal: {self.goal}
        Backstory: {self.backstory}
        Task: {task_description}
        Inputs: {inputs}
        Please respond according to your goal and task.
        """
        return local_llm(prompt)

# Agents
listener_agent = LocalAgent(
    name="Listener",
    role="Listener",
    goal="Receive messages and forward for analysis",
    backstory="A friendly assistant who listens carefully and passes messages for analysis."
)

analyzer_agent = LocalAgent(
    name="Analyzer",
    role="Sentiment Analyzer",
    goal="Analyze messages for sentiment and distress level",
    backstory="An expert analyst who identifies emotional tone and signs of distress."
)

# Tasks
class LocalTask:
    def __init__(self, name, description, agent, expected_output):
        self.name = name
        self.description = description
        self.agent = agent
        self.expected_output = expected_output

    def execute(self, inputs=None):
        print(f"\nExecuting Task: {self.name}")
        result = self.agent.run(self.description, inputs)
        print(f"Task Output: {result}")
        return result

# Define tasks for each agent
listening_task = LocalTask(
    name="Listening Task",
    description="Listen to user message and pass it to the sentiment analyzer.",
    agent=listener_agent,
    expected_output="Message forwarded."
)

sentiment_analysis_task = LocalTask(
    name="Sentiment Analysis Task",
    description="Analyze the message sentiment and detect signs of distress.",
    agent=analyzer_agent,
    expected_output="Sentiment (positive/negative/neutral) and distress level."
)

# Simulated Workflow
def run_workflow(user_message):
    print(f"User Message: {user_message}")
    forwarded_message = listening_task.execute(inputs={"message": user_message})
    analysis_result = sentiment_analysis_task.execute(inputs={"message": forwarded_message})
    print("\nFinal Analysis Result:")
    print(analysis_result)

# Example Run
user_message = "I'm feeling really overwhelmed and anxious today."
run_workflow(user_message)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User Message: I'm feeling really overwhelmed and anxious today.

Executing Task: Listening Task


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Task Output: 
        Role: Listener
        Goal: Receive messages and forward for analysis
        Backstory: A friendly assistant who listens carefully and passes messages for analysis.
        Task: Listen to user message and pass it to the sentiment analyzer.
        Inputs: {'message': "I'm feeling really overwhelmed and anxious today."}
        Please respond according to your goal and task.
        """
        # TODO: Implement the task
        pass
    
    def analyze_sentiment(self, message):
        """
        Task: Analyze the sentiment of a message
        Inputs: {'message': "I'm feeling really overwhelmed and anxious today."}
        Outputs: {'sentiment': "Negative"}
        """
        # TODO: Implement the task
        pass
    
    def respond_to_user(self, message):
        """
        Task: Respond to the user based on the sentiment analysis
        Inputs: {'message': "I'm feeling really overwhelmed and anxious today."}
        Outputs: {'response': "I'm sorry t

In [1]:
pip install gradio

In [10]:
import gradio as gr

def mental_health_chat(message):
    result = message_analysis_crew.kickoff(inputs={"message": message})
    return result['final_analysis']

gr.Interface(
    fn=mental_health_chat,
    inputs=gr.Textbox(label="How are you feeling today?"),
    outputs=gr.Textbox(label="AI Support Response"),
    title="🧠 Mental Health Companion Agent",
    description="Your friendly AI agent to provide emotional support and mental health guidance."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a5c7b5596aa7e159f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
